In [3]:
!pip install tqdm opencv-contrib-python matplotlib numpy --quiet

In [4]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Prepare the Dataset 

In [5]:
# Dataset directory path
DATASET_DIR = "./brandenburg_gate"

# Check if the dataset directory exists
if not os.path.exists(DATASET_DIR):
    print("Dataset directory not found. Please make sure the dataset is already extracted.")
else:
    print("Dataset found. Proceeding with feature matching...")

# load the image pairs
image_files = [os.path.join(DATASET_DIR, f) for f in os.listdir(DATASET_DIR) if f.endswith('.jpg')]
image_pairs = [(image_files[i], image_files[i + 1]) for i in range(0, min(len(image_files) - 1, 100), 2)]

Dataset found. Proceeding with feature matching...


# Feature Detection and Matching Functions

In [6]:
def match_features_and_error(img1_path, img2_path, method="SIFT", matcher_type="BF", visualize=False):
    img1 = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)
    if img1 is None or img2 is None:
        return None, None

    if method == "SIFT":
        feature_extractor = cv2.SIFT_create()
    elif method == "ORB":
        feature_extractor = cv2.ORB_create()
    elif method == "AKAZE":
        feature_extractor = cv2.AKAZE_create()
    elif method == "BRISK":
        feature_extractor = cv2.BRISK_create()
    elif method == "KAZE":
        feature_extractor = cv2.KAZE_create()
    else:
        return None, None

    keypoints1, descriptors1 = feature_extractor.detectAndCompute(img1, None)
    keypoints2, descriptors2 = feature_extractor.detectAndCompute(img2, None)

    if descriptors1 is None or descriptors2 is None:
        return None, None

    if matcher_type == "BF":
        matcher = cv2.BFMatcher()
        matches = matcher.knnMatch(descriptors1, descriptors2, k=2)
    elif matcher_type == "FLANN":
        if descriptors1.dtype != np.float32:
            descriptors1 = np.float32(descriptors1)
            descriptors2 = np.float32(descriptors2)
        index_params = dict(algorithm=1, trees=5)
        search_params = dict(checks=50)
        matcher = cv2.FlannBasedMatcher(index_params, search_params)
        matches = matcher.knnMatch(descriptors1, descriptors2, k=2)
    else:
        return None, None

    good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]

    if len(good_matches) < 4:
        return len(good_matches), None

    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    if H is not None:
        projected_pts = cv2.perspectiveTransform(src_pts, H)
        error = np.sqrt(np.sum((projected_pts - dst_pts) ** 2, axis=2)).mean()
    else:
        error = None

    return len(good_matches), error

# Main

In [ ]:
results = {"method": [], "matcher_type": [], "matches": [], "error": []}

for img1_path, img2_path in tqdm(image_pairs, desc="Matching Features"):
    for method in ["SIFT", "ORB", "AKAZE", "BRISK", "KAZE"]:
        for matcher_type in ["BF", "FLANN"]:
            matches, error = match_features_and_error(img1_path, img2_path, method, matcher_type)
            if matches is not None:
                results["method"].append(method)
                results["matcher_type"].append(matcher_type)
                results["matches"].append(matches)
                results["error"].append(error if error is not None else np.nan)

Matching Features:  26%|██▌       | 13/50 [00:32<01:28,  2.39s/it]

# Plotting the results

In [ ]:
detectors = ["SIFT", "ORB", "AKAZE", "BRISK", "KAZE"]
avg_errors = {method: {"BF": 0, "FLANN": 0} for method in detectors}
avg_matches = {method: {"BF": 0, "FLANN": 0} for method in detectors}
count = {method: {"BF": 0, "FLANN": 0} for method in detectors}

for method, matcher, matches, error in zip(results["method"], results["matcher_type"], results["matches"], results["error"]):
    avg_errors[method][matcher] += error if not np.isnan(error) else 0
    avg_matches[method][matcher] += matches
    count[method][matcher] += 1

for method in detectors:
    for matcher in ["BF", "FLANN"]:
        if count[method][matcher] > 0:
            avg_errors[method][matcher] /= count[method][matcher]
            avg_matches[method][matcher] /= count[method][matcher]

print("\nAverage Results:")
for method in detectors:
    for matcher in ["BF", "FLANN"]:
        print(f"Method: {method}, Matcher: {matcher}, Avg Matches: {avg_matches[method][matcher]:.2f}, Avg Error: {avg_errors[method][matcher]:.2f}")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14, 6))
x = np.arange(len(detectors))
width = 0.35

errors_bf = [avg_errors[method]["BF"] for method in detectors]
errors_flann = [avg_errors[method]["FLANN"] for method in detectors]
ax[0].bar(x - width/2, errors_bf, width, label='BF')
ax[0].bar(x + width/2, errors_flann, width, label='FLANN')
ax[0].set_ylabel('Average Reprojection Error (pixels)')
ax[0].set_title('Average Reprojection Error by Detector & Matcher')
ax[0].set_xticks(x)
ax[0].set_xticklabels(detectors)
ax[0].legend()

matches_bf = [avg_matches[method]["BF"] for method in detectors]
matches_flann = [avg_matches[method]["FLANN"] for method in detectors]
ax[1].bar(x - width/2, matches_bf, width, label='BF')
ax[1].bar(x + width/2, matches_flann, width, label='FLANN')
ax[1].set_ylabel('Average Number of Matches')
ax[1].set_title('Average Number of Matches by Detector & Matcher')
ax[1].set_xticks(x)
ax[1].set_xticklabels(detectors)
ax[1].legend()

plt.tight_layout()
plt.show()